# Stock Market Analysis Project
Gets the stock market tickers from the S&P500 and uses web scraping/parsing tools to get financial info from yahoo finance. 

In [2]:
# Imports
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import sys
import pandas as pd
from lxml import html  
import requests
from time import sleep
import json
import argparse
from collections import OrderedDict
from time import sleep
from collections import defaultdict
import csv
#import urllib
#urrlib.disable_warnings()

In [3]:
# Functions that get financial data 


def parse(ticker):
    '''
    parse: This function returns the summary info on the yahoo finance page for "ticker". 
    The information returned is in the form of a dictionary. 
    '''
    url = "https://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    response = requests.get(url, verify=False)
    #print ("Parsing %s"%(url))
    sleep(4)
    parser = html.fromstring(response.text)
    summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_json_response = requests.get(other_details_json_link)
    try:
        json_loaded_summary =  json.loads(summary_json_response.text)
        y_Target_Est = json_loaded_summary["quoteSummary"]["result"][0]["financialData"]["targetMeanPrice"]['raw']
        earnings_list = json_loaded_summary["quoteSummary"]["result"][0]["calendarEvents"]['earnings']
        eps = json_loaded_summary["quoteSummary"]["result"][0]["defaultKeyStatistics"]["trailingEps"]['raw']
        datelist = []
        for i in earnings_list['earningsDate']:
            datelist.append(i['fmt'])
        earnings_date = ' to '.join(datelist)
        for table_data in summary_table:
            raw_table_key = table_data.xpath('.//td[contains(@class,"C(black)")]//text()')
            raw_table_value = table_data.xpath('.//td[contains(@class,"Ta(end)")]//text()')
            table_key = ''.join(raw_table_key).strip()
            table_value = ''.join(raw_table_value).strip()
            summary_data.update({table_key:table_value})
        summary_data.update({'1y Target Est':y_Target_Est,'EPS (TTM)':eps,'Earnings Date':earnings_date,'ticker':ticker,'url':url})
        return summary_data
    except:
        print ("Failed to parse json response")
    return {"error":"Failed to parse json response"}


def periodic_figure_values(soup, yahoo_figure):
    '''
    periodic_figure_values: Call this function to obtain financial data from a company's financial statements.
        Args: 
            soup: use the function financials_soup("ticker", "is" or "bs" or "cf") to get the correct soup 
            yahoo_figure: The name of the information you want from the financial statement. Ex: Total Current Assets
        Return:
            This function normally returns a list of 4 elements, with numbers pertaining to the last 4 years
    '''
    values = []
    pattern = re.compile(yahoo_figure)

    title = soup.find("strong", text=pattern)    # works for the figures printed in bold
    if title:
        row = title.parent.parent
    else:
        title = soup.find("td", text=pattern)    # works for any other available figure
        if title:
            row = title.parent
        else:
            sys.exit("Invalid figure '" + yahoo_figure + "' passed.")

    cells = row.find_all("td")[1:]    # exclude the <td> with figure name
    for cell in cells:
        if cell.text.strip() != yahoo_figure:    # needed because some figures are indented
            str_value = cell.text.strip().replace(",", "").replace("(", "-").replace(")", "")
            if str_value == "-":
                str_value = 0
            value = int(float(str_value)) * 1000
            values.append(value)

    return values


def get_key_statistic(soup, name):
    value = 0
    pattern = re.compile(yahoo_figure)

    title = soup.find("strong", text=pattern)    # works for the figures printed in bold
    if title:
        row = title.parent.parent
    else:
        title = soup.find("td", text=pattern)    # works for any other available figure
        if title:
            row = title.parent
        else:
            sys.exit("Invalid figure '" + yahoo_figure + "' passed.")

    cells = row.find_all("td")[1:]    # exclude the <td> with figure name
    for cell in cells:
        if cell.text.strip() != yahoo_figure:    # needed because some figures are indented
            str_value = cell.text.strip().replace(",", "").replace("(", "-").replace(")", "")
            if str_value == "-":
                str_value = 0
            value = int(str_value) * 1000
            values.append(value)

    return values


def financials_soup(ticker_symbol, statement="is", quarterly=False):
    '''
    financials_soup: Gets the soup corresponding to the company and the financial statement you want. 
    This is used in the first arg for periodic_figure_values. 
    '''
    if statement == "is" or statement == "cf":
        url = "https://finance.yahoo.com/q/" + statement + "?s=" + ticker_symbol
        if not quarterly:
            url += "&annual"
        return BeautifulSoup(requests.get(url).text, "html.parser")
    if statement == "bs":
        url = "https://finance.yahoo.com/quote/" + ticker_symbol + "/balance-sheet?p=" + ticker_symbol
        if not quarterly:
            url += "&annual"
        return BeautifulSoup(requests.get(url).text, "html.parser")
    if statement == "ks":
        url = "https://finance.yahoo.com/quote/" + ticker_symbol + "/key-statistics?p=" + ticker_symbol
    return sys.exit("Invalid financial statement code '" + statement + "' passed.")

## Functions that calculate metrics

In [46]:
# Higher the better, preferably greater than 2
def get_current_ratio(ticker):
    try:
        total_current_assets = periodic_figure_values(financials_soup(ticker, "bs"), "Total Current Assets")
        total_current_liabilities = periodic_figure_values(financials_soup(ticker, "bs"), "Total Current Liabilities")
        cur_ratio = np.divide(total_current_assets, total_current_liabilities)
    except:
        print("Could not calculate the current ratio for " + ticker)
    return cur_ratio

#def get_shares_outstanding(ticker):
    
def get_current_assets_per_share(ticker):
    total_current_assets = periodic_figure_values(financials_soup(ticker, "bs"), "Total Current Assets")
    total_current_liabilities = periodic_figure_values(financials_soup(ticker, "bs"), "Total Current Liabilities")
    net_income = periodic_figure_values(financials_soup(ticker, "is"), "Net Income")
    try:
        shares_outstanding = np.divide(net_income, parse(ticker)['EPS (TTM)'])
        return np.divide(np.subtract(total_current_assets, total_current_liabilities), shares_outstanding)
    except:
        print("Could not calculate current assets per share for " + ticker)
        return 0

# The lower the better
def get_debt_ratio(ticker):
    try:
        total_assets = periodic_figure_values(financials_soup(ticker, "bs"), "Total Assets")
        total_liabilities = periodic_figure_values(financials_soup(ticker, "bs"), "Total Liabilities")
    except:
        print("Could not calculate debt ratio for " + ticker)
    return np.divide(total_liabilities, total_assets)

def get_book_value_per_share(ticker):
    try:
        total_assets = periodic_figure_values(financials_soup(ticker, "bs"), "Total Assets")
        total_liabilities = periodic_figure_values(financials_soup(ticker, "bs"), "Total Liabilities")
        net_income = periodic_figure_values(financials_soup(ticker, "is"), "Net Income")
        eps = parse(ticker)['EPS (TTM)']
        shares_outstanding = np.divide(net_income, eps)
    except:
        print("Could not calculate the book value per share for " + ticker)
    return np.divide(np.subtract(total_assets, total_liabilities), shares_outstanding)

def get_price_to_book_value(ticker):
    try:
        open_price = float(parse(ticker)['Open'])
        #print("open price")
        #print(open_price)
        bvps = float(get_book_value_per_share(ticker)[0])
        #print("bvps")
        #print(bvps)
    except:
        print("Could not calculate the price to book value for " + ticker)
    return np.divide(open_price , bvps)

def get_altman_zscore(ticker):
    # A = working capital / total assets
    total_cur_assets = periodic_figure_values(financials_soup(ticker, "bs"), "Total Current Assets")[0]
    total_cur_liabilities = periodic_figure_values(financials_soup(ticker, "bs"), "Total Current Liabilities")[0]
    total_assets = periodic_figure_values(financials_soup(ticker, "bs"), "Total Assets")[0]
    a = (total_cur_assets - total_cur_liabilities) / total_assets
    # B = retained earnings / total assets
    net_income = periodic_figure_values(financials_soup(ticker, "is"), "Net Income")[0]
    try:
        dividends_paid = periodic_figure_values(financials_soup(ticker, "cf"), "Dividends Paid")[0]
        retained_earnings = net_income + dividends_paid
    except:
        retained_earnings = net_income
    b = retained_earnings / total_assets
    # C = earnings before interest and tax / total assets
    operating_income = periodic_figure_values(financials_soup(ticker, "is"), "Earnings Before Interest and Taxes")[0]
    c = operating_income / total_assets
    # D = market value of equity / total liabilities
    market_cap = parse(ticker)["Market Cap"] 
    if market_cap[len(market_cap) - 1] == "B":
        market_cap = float(market_cap[0:len(market_cap) - 1]) * 1000000
    elif market_cap[len(market_cap) - 1] == "M":
        market_cap = float(market_cap[0:len(market_cap) - 1]) * 1000
    elif market_cap[len(market_cap) - 1] == "T":
        market_cap = float(market_cap[0:len(market_cap) - 1]) * 1000000000
    else:
        print("Error: Market Cap is " + market_cap[len(market_cap) - 1] + ". Expected M or B.")
    d = market_cap / periodic_figure_values(financials_soup(ticker, "bs"), "Total Liabilities")[0]
    # E = sales / total assets
    e = periodic_figure_values(financials_soup(ticker, "is"), "Total Revenue")[0] / total_assets
    return 1.2 * a + 1.4 * b + 3.3 * c + 0.6 * d + 1.0 * e

def get_earning_growth_yoy(ticker):
    try:
        net_income = periodic_figure_values(financials_soup(ticker, "is"), "Net Income")
        if net_income[0] < 0 and net_income[1] < 0:
            return -1 * (net_income[0] - net_income[1]) / net_income[1]
        elif net_income[1] < 0 and net_income[0] > 0:
            return -1 * (net_income[0] - net_income[1]) / net_income[1]
        else:
            return (net_income[0] - net_income[1]) / net_income[1]
    except: 
        print("Could not calculate the earning growth for " + ticker)
    
def get_dividend_yield(ticker):
    return parse(ticker)['Forward Dividend & Yield']

#def get_pe_ratio(ticker):
    

In [82]:
print(get_current_ratio("AMZN"))
print(get_debt_ratio("AMZN"))
print(get_current_assets_per_share("AMZN"))
print(get_book_value_per_share("AMZN"))
print(get_dividend_yield("COST"))

[ 1.0399772   1.04484663  1.05364889  1.11527644]
[ 0.78898028  0.76877053  0.79328772  0.80293551]


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[   9.63365579   10.46480599   38.51658725 -169.65239004]
AMZN


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[ 115.35824036  102.70421552  283.55665772 -562.76600415]


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2.28 (0.97%)


In [21]:
def analyze(ticker):
    cur_ratio = get_current_ratio(ticker)
    debt_ratio = get_debt_ratio(ticker)
    assets_per_share = get_current_assets_per_share(ticker)
    book_value = get_book_value_per_share(ticker)
    earning_growth_yoy = get_earning_growth_yoy(ticker)
    print("ANALYSIS FOR " + ticker)
    print("Book Value: " + str(book_value[0]))
    print("Current Ratio: " + str(cur_ratio[0]))
    print("Debt Ratio: " + str(debt_ratio[0]))
    print("Assets Per Share: " + str(assets_per_share[0]))
    print("Earnings growth (YOY): " + str(earning_growth_yoy))
    print("Forward Dividend & Yield: " + get_dividend_yield(ticker))
    
    

In [24]:
# Get S&P500 Tickers with Industry
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table = data[0]
sliced_table = table[1:]
header = table.iloc[0]
corrected_table = sliced_table.rename(columns=header)
tickers = corrected_table['Symbol'].tolist() # Gets a list of the tickers in the S&P500
industries = corrected_table['GICS Sub Industry'].tolist() # Gets a list of the industries in the S&P500

# Returns the industry of a company given its ticker
def get_company_industry(ticker):
    for i in range(len(corrected_table["Symbol"].tolist())):
        if ticker.lower() == corrected_table["Symbol"].tolist()[i].lower():
            return corrected_table["GICS Sub Industry"].tolist()[i]
    return "failed"

In [48]:
# Get companies
#tickers_full = np.genfromtxt(("\t".join(i) for i in csv.reader(open('companylist.csv'))), delimiter="\t")

with open('companylist.csv', newline='') as f:
    reader = csv.reader(f)
    company_matrix = np.array(list(reader))
    company_matrix = np.delete(company_matrix, (0), axis=0)
    
tickers_full = company_matrix[:,0]
industry = company_matrix[:,7]

company_industry = defaultdict(list)
for i in range(len(tickers_full)):
    if industry[i] in company_industry:
        company_industry[industry[i]].append(tickers_full[i])
    else:
        company_industry[industry[i]] = [tickers_full[i]]
print(company_industry)

defaultdict(<class 'list'>, {'Medical/Nursing Services': ['YI', 'ADUS', 'AMED', 'BIOS', 'ESRX', 'KANG', 'LHCG', 'PETS', 'WBA'], 'Property-Casualty Insurers': ['PIH', 'PIHPP', 'AMBC', 'AMBCW', 'AMSF', 'AFSI', 'ACGL', 'ACGLO', 'ACGLP', 'AFH', 'AFHBL', 'CINF', 'CNFR', 'CNFRL', 'DGICA', 'DGICB', 'EMCI', 'ESGR', 'ESGRP', 'FNHC', 'GBLI', 'GBLIL', 'GBLIZ', 'GLRE', 'HALL', 'ICCH', 'JRVR', 'KINS', 'KNSL', 'MHLD', 'NGHC', 'NGHCN', 'NGHCO', 'NGHCP', 'NGHCZ', 'NODK', 'NMIH', 'OXBR', 'OXBRW', 'PTVCA', 'PTVCB', 'SAFT', 'SIGI', 'STFC', 'NAVG', 'TIPT', 'UNAM', 'UFCS', 'UIHC'], 'Finance/Investors Services': ['TURN', 'LYL', 'LIVE'], 'Other Specialty Stores': ['FLWS', 'AMRK', 'BGFV', 'PRSS', 'DOTA', 'DOTAR', 'DOTAU', 'DOTAW', 'EZPW', 'FCFS', 'HIBB', 'IAC', 'JD', 'KIRK', 'ODP', 'ORLY', 'SECO', 'SPWH', 'LOVE', 'TITN', 'PRTS', 'ULTA', 'WINA', 'ZAGG'], 'Savings Institutions': ['FCCY', 'ANCX', 'BCTF', 'BFIN', 'BCBP', 'BNCL', 'BYFC', 'BRKL', 'CFFN', 'CARV', 'CBMB', 'CFBK', 'CZWI', 'CVLY', 'CLBK', 'CFBI', 'DCOM

In [12]:
company_industry = defaultdict(list)
for i in range(len(tickers)):
    if industries[i] in company_industry:
        company_industry[industries[i]].append(tickers[i])
    else:
        company_industry[industries[i]] = [tickers[i]]
print(company_industry)

defaultdict(<class 'list'>, {'Industrial Conglomerates': ['MMM', 'GE', 'HON', 'ROP'], 'Health Care Equipment': ['ABT', 'ABMD', 'A', 'BAX', 'BDX', 'BSX', 'DHR', 'EW', 'HOLX', 'IDXX', 'ISRG', 'JNJ', 'MDT', 'PKI', 'RMD', 'SYK', 'TMO', 'VAR', 'ZBH'], 'Pharmaceuticals': ['ABBV', 'AGN', 'LLY', 'MRK', 'MYL', 'NKTR', 'PRGO', 'PFE', 'ZTS'], 'IT Consulting & Other Services': ['ACN', 'CTSH', 'DXC', 'IT', 'IBM'], 'Interactive Home Entertainment': ['ATVI', 'EA', 'TTWO'], 'Application Software': ['ADBE', 'ANSS', 'ADSK', 'CDNS', 'ORCL', 'SYMC', 'SNPS'], 'Semiconductors': ['AMD', 'ADI', 'AVGO', 'INTC', 'MCHP', 'MU', 'NVDA', 'QRVO', 'QCOM', 'SWKS', 'TXN', 'XLNX'], 'Automotive Retail': ['AAP'], 'Independent Power Producers & Energy Traders': ['AES', 'NRG'], 'Managed Health Care': ['AET', 'ANTM', 'CNC', 'CI', 'HUM', 'UNH', 'WCG'], 'Asset Management & Custody Banks': ['AMG', 'AMP', 'BK', 'BLK', 'BEN', 'IVZ', 'NTRS', 'STT', 'TROW'], 'Life & Health Insurance': ['AFL', 'BHF', 'MET', 'PFG', 'PRU', 'TMK', 'UNM

In [41]:
def get_industry_averages(industry_dict):
    industry_current_ratio = {}
    industry_current_assets_per_share = {}
    industry_debt_ratio = {}
    industry_book_value_per_share = {}
    industry_earnings_growth_yoy = {}
    #industry_dividend_yield = {}
    for key in industry_dict.keys():
        current_ratio_av = 0
        current_assets_per_share_av = 0
        debt_ratio_av = 0
        book_value_per_share_av = 0
        earnings_growth_yoy_av = 0
        #dividend_yield_av = 0
        tickers_not_added = 0
        for ticker in industry_dict[key]:
            cur_ratio = get_current_ratio(ticker)
            if len(cur_ratio) == 0:
                add_to_averages = False
                tickers_not_added += 1
                break
            current_ratio_av += get_current_ratio(ticker)[0]
            cur_assets_per_share = get_current_assets_per_share(ticker)
            '''if len(cur_assets_per_share) < 4:
                add_to_averages = False
                tickers_not_added += 1
                break
            else:
                current_assets_per_share_av += cur_assets_per_share[0]
            '''
            try: 
                current_assets_per_share_av += cur_assets_per_share[0]
            except: 
                add_to_averages = False
                tickers_not_added += 1
                break
            debt_ratio_av += get_debt_ratio(ticker)[0]
            bvps = get_book_value_per_share(ticker)[0]
            print("book value per share: " + str(bvps))
            book_value_per_share_av += bvps
            earnings_growth_yoy_av += get_earning_growth_yoy(ticker)
            #dividend_yield_av += get_dividend_yield(ticker)
        #TODO: Check if the division by 0
        if (len(industry_dict[key]) - tickers_not_added) == 0:
            break;
        industry_current_ratio[key] = current_ratio_av / (len(industry_dict[key]) - tickers_not_added)
        industry_current_assets_per_share[key] = current_assets_per_share_av / (len(industry_dict[key]) - tickers_not_added)
        industry_debt_ratio[key] = debt_ratio_av / (len(industry_dict[key]) - tickers_not_added)
        industry_book_value_per_share[key] = book_value_per_share_av / (len(industry_dict[key]) - tickers_not_added)
        industry_earnings_growth_yoy[key] = earnings_growth_yoy_av / (len(industry_dict[key]) - tickers_not_added)
        #industry_dividend_yield[key] = dividend_yield_av / len(industsry_dict[key])
    return [industry_current_ratio, industry_current_assets_per_share, industry_debt_ratio, industry_book_value_per_share,industry_earnings_growth_yoy]#, industry_dividend_yield ]

In [49]:
industry_averages = get_industry_averages(company_industry)
%tb

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Failed to parse json response
Could not calculate current assets per share for YI


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


book value per share: 15.759857142857143


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


book value per share: 4.2342953679619555


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


book value per share: 10.699954116638079
Could not calculate the current ratio for BLIN          


UnboundLocalError: local variable 'cur_ratio' referenced before assignment

In [24]:
print(industry_averages)

[{'Industrial Conglomerates': 1.7200425152843348, 'Health Care Equipment': 2.2518869079741397, 'Pharmaceuticals': 2.400151267157737, 'IT Consulting & Other Services': 1.554371715684612, 'Interactive Home Entertainment': 1.8617676930527631, 'Application Software': 1.8085843138944175, 'Semiconductors': 3.9100322019355396, 'Automotive Retail': 1.559408257873272, 'Independent Power Producers & Energy Traders': 1.1962011167261053, 'Managed Health Care': 1.1465796052782662, 'Asset Management & Custody Banks': 3.7145084825912247, 'Life & Health Insurance': 2.2786661789894374, 'Industrial Gases': 1.6772080735520536, 'Internet Services & Infrastructure': 2.1944724111692602, 'Airlines': 0.61736169323599954, 'Specialty Chemicals': 1.8515889952659581, 'Office REITs': 3.5482943954945076, 'Biotechnology': 3.2361879259361426, 'Health Care Supplies': 2.2761926605887788, 'Building Products': 2.4624523102639033, 'Data Processing & Outsourced Services': 1.5664104703827437, 'Electric Utilities': 0.6374332

In [9]:
industry_averages = [{'Industrial Conglomerates': 1.8332163260006791, 'Health Care Equipment': 2.2538307538356617, 'Pharmaceuticals': float('nan'), 'IT Consulting & Other Services': 1.5329954640851751, 'Home Entertainment Software': 1.8617676930527631, 'Application Software': float('nan'), 'Semiconductors': 3.9100322019355396, 'Automotive Retail': 1.559408257873272, 'Independent Power Producers & Energy Traders': float('nan'), 'Managed Health Care': 1.1430793327046855, 'Asset Management & Custody Banks': 3.7042987746103333, 'Life & Health Insurance': float('nan'), 'Industrial Gases': 1.6772080735520536, 'Internet Software & Services': 2.732413250697459, 'Airlines': 0.61736169323599954, 'Specialty Chemicals': 1.8515889952659581, 'Office REITs': 3.5482943954945076, 'Biotechnology': 3.3531134585041329, 'Health Care Supplies': 1.5768736453975796, 'Building Products': 2.4624523102639033, 'Data Processing & Outsourced Services': float('inf'), 'Electric Utilities': 0.63779611123100288, 'Property & Casualty Insurance': 0.67182882825368995, 'Tobacco': 0.99620698097911875, 'Internet & Direct Marketing Retail': 1.7488471725115362, 'Multi-Utilities': 0.80435039024132893, 'Consumer Finance': 1.1371411720259779, 'Specialized REITs': 1.1805296464673631, 'Water Utilities': 0.30967741935483872, 'Health Care Distributors': float('inf'), 'Electrical Components & Equipment': 1.760146566451259, 'Electronic Components': 2.8493280362984423, 'Oil & Gas Exploration & Production': 1.2605093879113911, 'Oil & Gas Refining & Marketing': float('nan'), 'Insurance Brokers': 1.1262116075084367, 'Residential REITs': 0.31588277393063469, 'Technology Hardware, Storage & Peripherals': 1.5005950354721882, 'Semiconductor Equipment': 3.436901667458752, 'Auto Parts & Equipment': 1.5272478663300877, 'Agricultural Products': 1.585123309466985, 'Aerospace & Defense': 1.7551770710472063, 'Communications Equipment': 2.1873487515031065, 'Multi-line Insurance': 1.4207696144619684, 'Integrated Telecommunication Services': 0.91379211143478967, 'Specialty Stores': 1.7306281861453094, 'Paper Packaging': float('nan'), 'Oil & Gas Equipment & Services': 1.9784549007553831, 'Metal & Glass Containers': 0.91502313123934742, 'Diversified Banks': 0.33354011653956367, 'Regional Banks': 0.063550561234666084, 'Multi-Sector Holdings': 0.0, 'Computer & Electronics Retail': float('nan'), 'Distillers & Vintners': 0.0, 'Air Freight & Logistics': 1.5494055595970722, 'Systems Software': 1.8626422262344189, 'Packaged Foods & Meats': 0.96975430640720783, 'Hotels, Resorts & Cruise Lines': 0.38440904300669604, 'Construction Machinery & Heavy Trucks': 1.8713136490337563, 'Financial Exchanges & Data': 1.2925681303930194, 'Real Estate Services': 1.1836221371518751, 'Broadcasting': 1.5793051359516617, 'Health Care Technology': 3.0144909732204486, 'Fertilizers & Agricultural Chemicals': 2.1506742176907077, 'Investment Banking & Brokerage': float('nan'), 'Cable & Satellite': 2.0988683651567066, 'Integrated Oil & Gas': 1.4580300433734539, 'Restaurants': 1.4192595064861586, 'Household Products': 1.1032485722217338, 'Diversified Support Services': 2.5534865351987626, 'Soft Drinks': 2.1935222149483331, 'Hypermarkets & Super Centers': 0.87483667426730305, 'Personal Products': 1.1973766234695107, 'Railroads': 0.89310130077940275, 'Industrial Machinery': 1.757005753757813, 'Drug Retail': 1.0419905922162349, 'Homebuilding': 6.2096348730006525, 'Health Care Facilities': 1.8228053676220715, 'Agricultural & Farm Machinery': 2.1420982676886271, 'General Merchandise Stores': float('nan'), 'Diversified Chemicals': 1.7476662860725092, 'Industrial REITs': 0.0, 'Health Care Services': 1.9824467040791742, 'Research & Consulting Services': float('nan'), 'Reinsurance': 2.2007805589303153, 'Retail REITs': 0.88048315891390727, 'Data Processing & Outsources Services': 0.8667315996915641, 'Electronic Equipment & Instruments': 3.5125426338138368, 'Construction & Engineering': 1.6820194959252943, 'Apparel Retail': float('nan'), 'Automobile Manufacturers': 1.0596180741439749, 'Copper': 2.1395394998015087, 'Consumer Electronics': 2.8527217566758702, 'Tires & Rubber': 1.2097512437810944, 'Apparel, Accessories & Luxury Goods': 2.35684775922901, 'Motorcycle Manufacturers': 1.2300610796758882, 'Leisure Products': 2.4097226297781607, 'Health Care REITs': 1.7445064662051379, 'Oil & Gas Drilling': 3.5868780579874269, 'Home Improvement Retail': 1.112511480344474, 'Hotel & Resort REITs': 4.3922261484098941, 'Life Sciences Tools & Services': 2.727174343092829, 'Advertising': 0.95319981350492711, 'Electronic Manufacturing Services': 5.1794200678634974, 'Life Sciences Tools & Service': 1.1880165289256199, 'Trucking': 1.4522283596790779, 'Oil & Gas Storage & Transportation': 0.64142081770926362, 'Food Retail': 0.78305275762485027, 'Home Furnishings': 1.6717184087742976, 'Distributors': 2.8895929914328309, 'Department Stores': float('nan'), 'Construction Materials': 2.6392524910594686, 'Casinos & Gaming': 1.2708052909869418}, {'Industrial Conglomerates': 10.22286751809389, 'Health Care Equipment': 7.972013569643547, 'Pharmaceuticals': 1.1024841239492074, 'IT Consulting & Other Services': -7.9169346925271551, 'Home Entertainment Software': 6.8845014126076522, 'Application Software': float('nan'), 'Semiconductors': 21.233878437055459, 'Automotive Retail': 29.559857204445798, 'Independent Power Producers & Energy Traders': float('nan'), 'Managed Health Care': 10.946240464763816, 'Asset Management & Custody Banks': -77.160498173460141, 'Life & Health Insurance': 23.434874483100845, 'Industrial Gases': 10.024139948495092, 'Internet Software & Services': 22.94494819618297, 'Airlines': -9.9072844239622064, 'Specialty Chemicals': 15.640824142772635, 'Office REITs': 8.8765319281792028, 'Biotechnology': 16.710715086410257, 'Health Care Supplies': 3.2659922287569318, 'Building Products': 4.8409941541776904, 'Data Processing & Outsourced Services': 121.38617456120883, 'Electric Utilities': -5.1709604599420791, 'Property & Casualty Insurance': -27.666958038070113, 'Tobacco': 1.1633909175453216, 'Internet & Direct Marketing Retail': 33.986228340608541, 'Multi-Utilities': -0.99532022296823264, 'Consumer Finance': -120.45020903888397, 'Specialized REITs': 1.7656436230768411, 'Water Utilities': -9.8711267605633797, 'Health Care Distributors': 13.960274672552563, 'Electrical Components & Equipment': 6.5821035751030461, 'Electronic Components': 12.065872395870606, 'Oil & Gas Exploration & Production': -2.6127225570971895, 'Oil & Gas Refining & Marketing': 11.49547314374591, 'Insurance Brokers': 4.3914899791211717, 'Residential REITs': -3.0040890230014305, 'Technology Hardware, Storage & Peripherals': 7.7735010930644712, 'Semiconductor Equipment': 14.77220828224959, 'Auto Parts & Equipment': 7.5761633663125441, 'Agricultural Products': 15.724641073657926, 'Aerospace & Defense': 25.368706518530104, 'Communications Equipment': 6.2628622174937432, 'Multi-line Insurance': -5.6020308474964082, 'Integrated Telecommunication Services': -0.6733566929466267, 'Specialty Stores': 5.5910479737748604, 'Paper Packaging': 3.3932381243293785, 'Oil & Gas Equipment & Services': 2.8671467552588599, 'Metal & Glass Containers': -1.1581289473684209, 'Diversified Banks': -231.08825614229048, 'Regional Banks': -339.09462042043918, 'Multi-Sector Holdings': 0.0, 'Computer & Electronics Retail': float('nan'), 'Distillers & Vintners': 0.0, 'Air Freight & Logistics': 7.8149618501274158, 'Systems Software': 7.3023656629428153, 'Packaged Foods & Meats': -1.1691311643498006, 'Hotels, Resorts & Cruise Lines': -10.086794913320782, 'Construction Machinery & Heavy Trucks': 48.214214567147437, 'Financial Exchanges & Data': 3.1272208677849003, 'Real Estate Services': 2.5814728646628819, 'Broadcasting': 6.5654965622612682, 'Health Care Technology': 4.590383220796836, 'Fertilizers & Agricultural Chemicals': -38.359781792819497, 'Investment Banking & Brokerage': 264.87489465344714, 'Cable & Satellite': 4.571590489180279, 'Integrated Oil & Gas': 2.9657532913141833, 'Restaurants': 2.2582464189430831, 'Household Products': 0.29350491125424932, 'Diversified Support Services': 6.6113367737144451, 'Soft Drinks': 4.7454630323018199, 'Hypermarkets & Super Centers': -1.7901329485939876, 'Personal Products': 1.670244339638292, 'Railroads': -1.0496760717962113, 'Industrial Machinery': 3.9664003555553471, 'Drug Retail': 0.75328263387249073, 'Homebuilding': 45.331589029030397, 'Health Care Facilities': 8.6680361903012919, 'Agricultural & Farm Machinery': 76.30099819377547, 'General Merchandise Stores': float('nan'), 'Diversified Chemicals': 11.151890560276133, 'Industrial REITs': 0.0, 'Health Care Services': -32.426330378469167, 'Research & Consulting Services': float('nan'), 'Reinsurance': 30.021646794664026, 'Retail REITs': -1.2013754044665081, 'Data Processing & Outsources Services': -6.6437003647662785, 'Electronic Equipment & Instruments': 8.1901561232198326, 'Construction & Engineering': 10.201045152938095, 'Apparel Retail': float('nan'), 'Automobile Manufacturers': 43.657503743782875, 'Copper': 5.6108442582552991, 'Consumer Electronics': 7.0030472908317805, 'Tires & Rubber': 3.0984712328767121, 'Apparel, Accessories & Luxury Goods': 12.030265154602464, 'Motorcycle Manufacturers': 4.0690115244777765, 'Leisure Products': 7.891296169064681, 'Health Care REITs': 0.20605798523936691, 'Oil & Gas Drilling': -28.873208105550475, 'Home Improvement Retail': 1.8120146575304705, 'Hotel & Resort REITs': 1.4795096322241681, 'Life Sciences Tools & Services': 13.482999614662511, 'Advertising': -2.330208440028251, 'Electronic Manufacturing Services': 18.476046918656355, 'Life Sciences Tools & Service': 2.4570000000000003, 'Trucking': 4.1478033232157356, 'Oil & Gas Storage & Transportation': -0.17427283963417348, 'Food Retail': -6.6344732662784542, 'Home Furnishings': 39.071716894308096, 'Distributors': 8.3038423461843589, 'Department Stores': float('nan'), 'Construction Materials': 17.076499539820734, 'Casinos & Gaming': 3.6607015703295875}, {'Industrial Conglomerates': 0.72479993152075839, 'Health Care Equipment': 0.51779255561153126, 'Pharmaceuticals': 0.25558346534109516, 'IT Consulting & Other Services': 0.63735192072527358, 'Home Entertainment Software': 0.52027392026685859, 'Application Software': float('nan'), 'Semiconductors': 0.44838712526486663, 'Automotive Retail': 0.59737387296206534, 'Independent Power Producers & Energy Traders': float('nan'), 'Managed Health Care': 0.68202050368549882, 'Asset Management & Custody Banks': 0.65221102397807473, 'Life & Health Insurance': 0.75955316272128559, 'Industrial Gases': 0.56465593394870361, 'Internet Software & Services': 0.63421097790254677, 'Airlines': 0.73854239419753998, 'Specialty Chemicals': 0.64757899204259339, 'Office REITs': 0.5439532115787663, 'Biotechnology': 0.46678284135544995, 'Health Care Supplies': 0.35348306398448831, 'Building Products': 0.61407101872566094, 'Data Processing & Outsourced Services': 0.72776988449703894, 'Electric Utilities': 0.73452868981431108, 'Property & Casualty Insurance': 0.77737686237631853, 'Tobacco': 0.94060126490003082, 'Internet & Direct Marketing Retail': 0.60360751342423047, 'Multi-Utilities': 0.72945169452498759, 'Consumer Finance': 0.87667836015568279, 'Specialized REITs': 0.65959075033440828, 'Water Utilities': 0.72359100708346169, 'Health Care Distributors': 0.71788832174610995, 'Electrical Components & Equipment': 0.53334622323433944, 'Electronic Components': 0.51111897050482746, 'Oil & Gas Exploration & Production': 0.5042917600106348, 'Oil & Gas Refining & Marketing': 0.50736947807157085, 'Insurance Brokers': 0.70449940438221981, 'Residential REITs': 0.50718743857598092, 'Technology Hardware, Storage & Peripherals': 0.74114894936607323, 'Semiconductor Equipment': 0.61509204698781206, 'Auto Parts & Equipment': 0.66004718804110629, 'Agricultural Products': 0.54019968470835522, 'Aerospace & Defense': 0.78862701419595416, 'Communications Equipment': 0.63811866470387268, 'Multi-line Insurance': 0.83179795635091569, 'Integrated Telecommunication Services': 0.73192309459380545, 'Specialty Stores': 0.75052406012896167, 'Paper Packaging': 0.40085088324330909, 'Oil & Gas Equipment & Services': 0.45754001695362706, 'Metal & Glass Containers': 0.76434271069951654, 'Diversified Banks': 0.89119086746239473, 'Regional Banks': 0.88093041517666537, 'Multi-Sector Holdings': 0.0, 'Computer & Electronics Retail': float('nan'), 'Distillers & Vintners': 0.0, 'Air Freight & Logistics': 0.65746431024842211, 'Systems Software': 0.65336974580575935, 'Packaged Foods & Meats': 0.6535432394414058, 'Hotels, Resorts & Cruise Lines': 0.64347885481349776, 'Construction Machinery & Heavy Trucks': 0.7388420361055823, 'Financial Exchanges & Data': 0.74120936172243201, 'Real Estate Services': 0.64475719337538084, 'Broadcasting': 0.90510003358441682, 'Health Care Technology': 0.26030020816745397, 'Fertilizers & Agricultural Chemicals': 0.56405909469542126, 'Investment Banking & Brokerage': 0.68103925172540014, 'Cable & Satellite': 0.6936149089995497, 'Integrated Oil & Gas': 0.43978419910745509, 'Restaurants': 0.96823441023847112, 'Household Products': 0.85259515218766435, 'Diversified Support Services': 0.44066931835563883, 'Soft Drinks': 0.61115753553348029, 'Hypermarkets & Super Centers': 0.6500064786719848, 'Personal Products': 0.58590000549467058, 'Railroads': 0.54277259015116153, 'Industrial Machinery': 0.61155978286553714, 'Drug Retail': 0.58771069732914893, 'Homebuilding': 0.50308931785093047, 'Health Care Facilities': 0.78508489895717748, 'Agricultural & Farm Machinery': 0.85446057917833951, 'General Merchandise Stores': float('nan'), 'Diversified Chemicals': 0.5635309325166642, 'Industrial REITs': 0.0, 'Health Care Services': 0.5544073057045652, 'Research & Consulting Services': float('nan'), 'Reinsurance': 0.6452481439307366, 'Retail REITs': 0.5742240600957752, 'Data Processing & Outsources Services': 0.6751718159849851, 'Electronic Equipment & Instruments': 0.34713842505371839, 'Construction & Engineering': 0.47729513867276463, 'Apparel Retail': float('nan'), 'Automobile Manufacturers': 0.8469053103149512, 'Copper': 0.69717441424052329, 'Consumer Electronics': 0.24106413639212337, 'Tires & Rubber': 0.71577590248476319, 'Apparel, Accessories & Luxury Goods': 0.54930288660531179, 'Motorcycle Manufacturers': 0.81506691486494287, 'Leisure Products': 0.72625019931878476, 'Health Care REITs': 0.5307814850792546, 'Oil & Gas Drilling': 0.35332317389411283, 'Home Improvement Retail': 0.90046537766056944, 'Hotel & Resort REITs': 0.38689814418883095, 'Life Sciences Tools & Services': 0.61029659849967144, 'Advertising': 0.83513810420923285, 'Electronic Manufacturing Services': 0.32242945889197633, 'Life Sciences Tools & Service': 0.63248614897546385, 'Trucking': 0.58808930724115849, 'Oil & Gas Storage & Transportation': 0.6230835342802109, 'Food Retail': 0.81436675000672099, 'Home Furnishings': 0.53895204240937034, 'Distributors': 0.55090098380761476, 'Department Stores': float('nan'), 'Construction Materials': 0.4782596335861361, 'Casinos & Gaming': 0.75669141160750208}, {'Industrial Conglomerates': 17.419408679529422, 'Health Care Equipment': 26.568652704776447, 'Pharmaceuticals': 14.678269345300794, 'IT Consulting & Other Services': 76.307213768480622, 'Home Entertainment Software': 15.431090970554841, 'Application Software': float('nan'), 'Semiconductors': 43.922738888708466, 'Automotive Retail': 51.855848245549467, 'Independent Power Producers & Energy Traders': float('nan'), 'Managed Health Care': 68.724115686028156, 'Asset Management & Custody Banks': 60.397250491546679, 'Life & Health Insurance': 72.318600569343062, 'Industrial Gases': 42.244670156801952, 'Internet Software & Services': 41.345305006951563, 'Airlines': 19.426424105639477, 'Specialty Chemicals': 42.325068793834639, 'Office REITs': 91.430499856562093, 'Biotechnology': 16.900534518104013, 'Health Care Supplies': 23.320921432469234, 'Building Products': 11.490075031694424, 'Data Processing & Outsourced Services': 22.383910073887442, 'Electric Utilities': 33.881005810702113, 'Property & Casualty Insurance': 105.0561653790259, 'Tobacco': 0.8729242878504655, 'Internet & Direct Marketing Retail': 106.02922884847627, 'Multi-Utilities': 26.237156503088134, 'Consumer Finance': 45.400228717463143, 'Specialized REITs': 30.566041241817473, 'Water Utilities': 33.119014084507043, 'Health Care Distributors': 14.317632487127797, 'Electrical Components & Equipment': 21.907244097516585, 'Electronic Components': 25.889326319350946, 'Oil & Gas Exploration & Production': 3.9384851645579171, 'Oil & Gas Refining & Marketing': 67.790418272078185, 'Insurance Brokers': 33.791928316845379, 'Residential REITs': 45.206638076305545, 'Technology Hardware, Storage & Peripherals': 31.603095273835482, 'Semiconductor Equipment': 9.5952459718408054, 'Auto Parts & Equipment': 17.824024509005294, 'Agricultural Products': 39.284878277153553, 'Aerospace & Defense': 30.473037767203298, 'Communications Equipment': 11.719548167729522, 'Multi-line Insurance': 66.062097891378585, 'Integrated Telecommunication Services': 22.00465063541748, 'Specialty Stores': 7.1926669723650773, 'Paper Packaging': 8.317759137686302, 'Oil & Gas Equipment & Services': 13.398672305655946, 'Metal & Glass Containers': 13.426331578947368, 'Diversified Banks': 51.792568054674327, 'Regional Banks': 52.66735142783503, 'Multi-Sector Holdings': 0.0, 'Computer & Electronics Retail': float('nan'), 'Distillers & Vintners': 0.0, 'Air Freight & Logistics': 25.159819758490585, 'Systems Software': 11.123309703590865, 'Packaged Foods & Meats': 24.721862306866527, 'Hotels, Resorts & Cruise Lines': 27.113947265529692, 'Construction Machinery & Heavy Trucks': 60.992868532363318, 'Financial Exchanges & Data': 26.367620351304478, 'Real Estate Services': 12.450013668679237, 'Broadcasting': 5.643873185637891, 'Health Care Technology': 13.809970605943864, 'Fertilizers & Agricultural Chemicals': -58.697258347689683, 'Investment Banking & Brokerage': 72.237281054111179, 'Cable & Satellite': 42.717462117161411, 'Integrated Oil & Gas': 62.272581427850518, 'Restaurants': 8.9707484516310885, 'Household Products': 4.3397925853831287, 'Diversified Support Services': 17.525943643405569, 'Soft Drinks': 7.8791113711926464, 'Hypermarkets & Super Centers': 20.651322844996653, 'Personal Products': 16.415826608815181, 'Railroads': 40.066445672696439, 'Industrial Machinery': 17.115353900416373, 'Drug Retail': 22.991667113037927, 'Homebuilding': 29.370054942896115, 'Health Care Facilities': 19.202846164455249, 'Agricultural & Farm Machinery': 28.339491246758062, 'General Merchandise Stores': float('nan'), 'Diversified Chemicals': 49.629744363654574, 'Industrial REITs': 0.0, 'Health Care Services': -42.662708643743223, 'Research & Consulting Services': float('nan'), 'Reinsurance': 91.675220450009377, 'Retail REITs': 24.146175429719836, 'Data Processing & Outsources Services': 44.752044339367735, 'Electronic Equipment & Instruments': 15.14212277216642, 'Construction & Engineering': 27.335902047627183, 'Apparel Retail': float('nan'), 'Automobile Manufacturers': -179.64409480224373, 'Copper': 11.039905372104487, 'Consumer Electronics': 17.344744940319877, 'Tires & Rubber': 14.25767123287671, 'Apparel, Accessories & Luxury Goods': 24.989656318172365, 'Motorcycle Manufacturers': 10.328479995553504, 'Leisure Products': 6.2889588393497551, 'Health Care REITs': 24.109610355468362, 'Oil & Gas Drilling': -134.97309701790198, 'Home Improvement Retail': 4.7643816534840946, 'Hotel & Resort REITs': 11.048546409807356, 'Life Sciences Tools & Services': 20.396187827628658, 'Advertising': 10.027286133523365, 'Electronic Manufacturing Services': 31.961076955786591, 'Life Sciences Tools & Service': 37.611000000000004, 'Trucking': 18.300434527287646, 'Oil & Gas Storage & Transportation': 1.6712696525283153, 'Food Retail': 14.873713605082054, 'Home Furnishings': 185.83716016727658, 'Distributors': 13.975851627825556, 'Department Stores': float('nan'), 'Construction Materials': 57.663846037867472, 'Casinos & Gaming': 12.124042023652922}, {'Industrial Conglomerates': 0.23586350543664966, 'Health Care Equipment': 1.2168703415351632, 'Pharmaceuticals': -0.39463333177288307, 'IT Consulting & Other Services': -3.102856223199346, 'Home Entertainment Software': 1.3131955352399378, 'Application Software': -0.0750387119169458, 'Semiconductors': 0.635822419945212, 'Automotive Retail': 1.0345566574271903, 'Independent Power Producers & Energy Traders': -0.1989247311827957, 'Managed Health Care': 1.7314515521814609, 'Asset Management & Custody Banks': 1.2249034760529998, 'Life & Health Insurance': 1.0849136200906722, 'Industrial Gases': 0.9400225771831371, 'Internet Software & Services': 0.8378191519597188, 'Airlines': 1.0602793697780433, 'Specialty Chemicals': 1.2487415294801019, 'Office REITs': -0.517465094551007, 'Biotechnology': 0.12845116705545445, 'Health Care Supplies': -0.3390412982363567, 'Building Products': 1.2744743456225605, 'Data Processing & Outsourced Services': 1.5519058347009649, 'Electric Utilities': 0.968652376940786, 'Property & Casualty Insurance': -2.461484495902262, 'Tobacco': 0.796303605147721, 'Internet & Direct Marketing Retail': 0.9096342307883869, 'Multi-Utilities': 1.2970752610614507, 'Consumer Finance': 0.8587207752763935, 'Specialized REITs': 1.336934660024726, 'Water Utilities': 0.9102564102564102, 'Health Care Distributors': 0.4162314782319822, 'Electrical Components & Equipment': 1.2619761206561495, 'Electronic Components': 0.32951611220309596, 'Oil & Gas Exploration & Production': -1.8216433499581686, 'Oil & Gas Refining & Marketing': 0.9209168235423406, 'Insurance Brokers': 0.8937868610469917, 'Residential REITs': 0.7804829848309871, 'Technology Hardware, Storage & Peripherals': 0.9490173575982045, 'Semiconductor Equipment': 1.430828664480067, 'Auto Parts & Equipment': 2.120754155232227, 'Agricultural Products': 1.2437888198757765, 'Aerospace & Defense': 0.865492363107537, 'Communications Equipment': 0.8061997395066917, 'Multi-line Insurance': 1.5450633191234475, 'Integrated Telecommunication Services': 2.2341445814090295, 'Specialty Stores': 1.0692244574889143, 'Paper Packaging': 0.8790507093857187, 'Oil & Gas Equipment & Services': -0.348768737550634, 'Metal & Glass Containers': 1.4285714285714286, 'Diversified Banks': 0.8664718881525305, 'Regional Banks': 1.3262294481086685, 'Multi-Sector Holdings': 0.0, 'Computer & Electronics Retail': 0.0, 'Distillers & Vintners': 0.0, 'Air Freight & Logistics': 1.2684507320832008, 'Systems Software': 0.7614731255493767, 'Packaged Foods & Meats': 1.4459209274156983, 'Hotels, Resorts & Cruise Lines': -30.567345252264708, 'Construction Machinery & Heavy Trucks': -4.826682975799456, 'Financial Exchanges & Data': 2.701540823086532, 'Real Estate Services': 1.1949820567609029, 'Broadcasting': 0.8434278350515464, 'Health Care Technology': 1.362136361636742, 'Fertilizers & Agricultural Chemicals': -1.2771294049801225, 'Investment Banking & Brokerage': 0.7344566500444125, 'Cable & Satellite': 1.196619659320218, 'Integrated Oil & Gas': -6.594835842598889, 'Restaurants': 2.476586571880507, 'Household Products': 1.1690126971174435, 'Diversified Support Services': 1.3872717964186008, 'Soft Drinks': 0.7004624619709331, 'Hypermarkets & Super Centers': 0.9392452063252456, 'Personal Products': 0.5119874336770974, 'Railroads': 2.7427233045534662, 'Industrial Machinery': 0.9646786498338017, 'Drug Retail': 1.1124769915124246, 'Homebuilding': 0.9196160730221621, 'Health Care Facilities': 0.9115575461595714, 'Agricultural & Farm Machinery': 1.419125862635557, 'General Merchandise Stores': 0.785726838372008, 'Diversified Chemicals': 0.9974030117609243, 'Industrial REITs': 0.0, 'Health Care Services': 1.744551835797428, 'Research & Consulting Services': 1.2538580063979063, 'Reinsurance': 0.4706888383931654, 'Retail REITs': 0.9521679395456176, 'Data Processing & Outsources Services': 1.6362169391115975, 'Electronic Equipment & Instruments': 0.643495012783119, 'Construction & Engineering': 1.2449822740365917, 'Apparel Retail': 0.39672782184578115, 'Automobile Manufacturers': 0.8456719047082331, 'Copper': -0.5294885177453027, 'Consumer Electronics': 1.3604854213079516, 'Tires & Rubber': 0.2842679127725857, 'Apparel, Accessories & Luxury Goods': 0.3561361430206693, 'Motorcycle Manufacturers': 0.7538083459989251, 'Leisure Products': -1.284857395745291, 'Health Care REITs': 1.2385254587221353, 'Oil & Gas Drilling': -2.4129647103227025, 'Home Improvement Retail': 1.0995158018968834, 'Hotel & Resort REITs': 0.7405966277561609, 'Life Sciences Tools & Services': 1.2811317050184559, 'Advertising': 0.9484980268858403, 'Electronic Manufacturing Services': 1.097566161063419, 'Life Sciences Tools & Service': 10.215384615384615, 'Trucking': 1.5882408757434794, 'Oil & Gas Storage & Transportation': -2.0210655641410415, 'Food Retail': 0.9652529381706694, 'Home Furnishings': 0.9219173327379773, 'Distributors': 1.1772570118147956, 'Department Stores': 1.2569558101472995, 'Construction Materials': 1.5407947306902599, 'Casinos & Gaming': 2.3219264333823877}]

In [40]:
def analyze(ticker, industry_averages):
    industry = get_company_industry(ticker)
    cur_ratio = get_current_ratio(ticker)
    cur_ratio_av = industry_averages[0][industry]
    debt_ratio = get_debt_ratio(ticker)
    debt_ratio_av = industry_averages[1][industry]
    assets_per_share = get_current_assets_per_share(ticker)
    assets_per_share_av = industry_averages[2][industry]
    book_value = get_book_value_per_share(ticker)
    price_to_book_val = get_price_to_book_value(ticker)
    book_value_av = industry_averages[3][industry]
    earning_growth_yoy = get_earning_growth_yoy(ticker)
    earning_growth_yoy_av = industry_averages[4][industry]
    altman_zscore = get_altman_zscore(ticker)
    print("ANALYSIS FOR " + ticker)
    print("Industry: " + industry)
    print("Book Value: " + str(book_value[0]))
    print("Price to book value ratio: " + str(price_to_book_val))
    print("Book value average for industry: " + str(book_value_av))
    print("Current Ratio: " + str(cur_ratio[0]))
    print("Current ratio average for industry: " + str(cur_ratio_av)) 
    print("Debt Ratio: " + str(debt_ratio[0]))
    print("Debt ratio average for industry: " + str(debt_ratio_av))
    print("Working Capital Per Share (current assets - current liabilities) / num shares: " + str(assets_per_share[0]))
    print("Working Capital per share average for industry: " + str(assets_per_share_av))
    print("Earnings growth (YOY): " + str(earning_growth_yoy))
    print("Earnings growth average for industry: " + str(earning_growth_yoy_av)) 
    dividend_yield_raw = get_dividend_yield(ticker)
    isPercent = False
    dividend_yield = ''
    for letter in dividend_yield_raw:
        if letter == "%":
            break;
        elif isPercent:
            dividend_yield += letter
        if letter == "(":
            isPercent = True
    dividend_yield = float(dividend_yield) / 100.0
    print("Forward Dividend & Yield: " + get_dividend_yield(ticker))
    print("Altman Zscore: " + str(altman_zscore))
    return [industry, cur_ratio, cur_ratio_av, debt_ratio, debt_ratio_av, assets_per_share, assets_per_share_av, 
           book_value, price_to_book_val, book_value_av, earning_growth_yoy, earning_growth_yoy_av, altman_zscore, 
           dividend_yield]

In [96]:
get_current_assets_per_share('TXT')[0]

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


25.116

In [38]:
analyze("COST", industry_averages)

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


open price
228.18


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


bvps
29.223111041207925


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1.03


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Forward Dividend & Yield: 2.28 (1.03%)
Altman Zscore: 3.951386135773019


['Hypermarkets & Super Centers',
 array([ 1.0182174 ,  0.98982566,  0.97707865,  1.01451116]),
 0.88903254422765388,
 array([ 0.67908401,  0.69518805,  0.62813979,  0.67159342]),
 -1.1613976723256061,
 array([ 0.80958478, -0.46500368, -1.0652904 ,  0.70635118]),
 0.64195445783473426,
 array([ 29.22311104,  28.9425608 ,  36.79877104,  31.91235782]),
 7.8082035714212674,
 21.324320701635987,
 0.17133382461311716,
 -0.046215827496142045,
 3.951386135773019]

In [124]:
s = 'abcd'
s[0:len(s) - 1]

'abc'

In [27]:
analyze("AAPL", industry_averages)

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


open price
216.88


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


bvps
30.60145159355546


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ANALYSIS FOR AAPL
Industry: Technology Hardware, Storage & Peripherals
Book Value: 30.6014515936
Price to book value ratio: 7.08724549674
Book value average for industry: 31.6030952738
Current Ratio: 1.27606284841
Current ratio average for industry: 1.50059503547
Debt Ratio: 0.642845153056
Debt ratio average for industry: 7.77350109306
Working Capital Per Share (current assets - current liabilities) / num shares: 6.35351033071
Working Capital per share average for industry: 0.741148949366
Earnings growth (YOY): 0.058309803664061986
Earnings growth average for industry: -0.05098264240179553


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Forward Dividend & Yield: 2.92 (1.37%)
Altman Zscore: 1.422113504604871


['Technology Hardware, Storage & Peripherals',
 array([ 1.27606285,  1.35266942,  1.10877062,  1.08011285]),
 1.5005950354721882,
 array([ 0.64284515,  0.60132241,  0.58892008,  0.51886007]),
 7.7735010930644712,
 array([ 6.35351033,  6.73171349,  1.81258538,  1.42004946]),
 0.74114894936607323,
 array([ 30.60145159,  30.98501679,  24.67394258,  31.16314315]),
 7.0872454967356528,
 31.603095273835482,
 0.058309803664061986,
 -0.05098264240179553,
 1.422113504604871]

In [19]:
notable_tickers = []
for ticker in tickers:
    try:
        analysis = analyze(ticker, industry_averages)
        current_ratio = float(analysis[1][0])
        current_ratio_av = float(analysis[2])
        current_ratio_cmp_to_av = float(current_ratio - current_ratio_av)
        price_to_book_ratio = float(analysis[8])
        earning_growth_yoy = float(analysis[10])
        earning_growth_yoy_av = float(analysis[11])
        earning_growth_cmp_to_av = earning_growth_yoy - earning_growth_yoy_av
        altman_zscore = analysis[12]
        dividend_yield = analysis[13]
        if current_ratio_cmp_to_av > 0.1 and price_to_book_ratio < 1 and altman_zscore > 3 and earning_growth_cmp_to_av > 0.1:
            notable_tickers.append(ticker) 
    except:
        print("Ticker: " + ticker + " did not work.")
    
print(notable_tickers)

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


open price
209.53


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


bvps
17.252786198398027


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

bvps
4.717774941995359


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ANALYSIS FOR ZTS
Industry: Pharmaceuticals
Book Value: 4.717774942
Price to book value ratio: 19.3057110862
Book value average for industry: 14.678269345300794
Current Ratio: 3.85466179159
Current ratio average for industry: nan
Debt Ratio: 0.791986955509
Debt ratio average for industry: 1.1024841239492074
Working Capital Per Share (current assets - current liabilities) / num shares: 8.24950232019
Working Capital per share average for industry: 0.25558346534109516
Earnings growth (YOY): 1.0525030525030525
Earnings growth average for industry: -0.39463333177288307


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Forward Dividend & Yield: 0.50 (0.56%)
Altman Zscore: 1.8662499141146323
[]


In [46]:
notable_tickers = []
csv = open("company_analysis.csv", "w")
csv.write("TICKER, CUR_RATIO, CUR_RATIO_AV, CUR_RATIO_CMP_TO_AV, PRICE_TO_BOOK, EARNING_GROWTH_YOY, EARNING_GROWTH_AV, EARNING_GROWTH_CMP_TO_AV, ALTMAN_ZSCORE, DIVIDEND_YIELD\n")
for ticker in tickers:
    try:
        analysis = analyze(ticker, industry_averages)
        current_ratio = float(analysis[1][0])
        current_ratio_av = float(analysis[2])
        current_ratio_cmp_to_av = float(current_ratio - current_ratio_av)
        price_to_book_ratio = float(analysis[8])
        earning_growth_yoy = float(analysis[10])
        earning_growth_yoy_av = float(analysis[11])
        earning_growth_cmp_to_av = earning_growth_yoy - earning_growth_yoy_av
        altman_zscore = analysis[12]
        dividend_yield = analysis[13]
        if current_ratio_cmp_to_av > 0.1 and price_to_book_ratio < 10 and altman_zscore > 1.3 and earning_growth_cmp_to_av > 0:
            notable_tickers.append(ticker)
        csv.write(ticker + ", " + str(current_ratio) + ", " + str(current_ratio_av) + ", " + str(current_ratio_cmp_to_av) 
                  + ", " + str(price_to_book_ratio) + ", " + str(earning_growth_yoy) + ", " + str(earning_growth_yoy_av)
                  + ", " + str(earning_growth_cmp_to_av) + ", " + str(altman_zscore) + ", " + str(dividend_yield) + "\n")
    except:
        print("Ticker: " + ticker + " did not work.")
    
print(notable_tickers)
csv.close()

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


open price
193.66


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


bvps
17.252786198398027


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

bvps
4.717774941995359


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ANALYSIS FOR ZTS
Industry: Pharmaceuticals
Book Value: 4.717774942
Price to book value ratio: 20.0009540854
Book value average for industry: 29.3538710233
Current Ratio: 3.85466179159
Current ratio average for industry: 2.40015126716
Debt Ratio: 0.791986955509
Debt ratio average for industry: 2.15584558649
Working Capital Per Share (current assets - current liabilities) / num shares: 8.24950232019
Working Capital per share average for industry: 0.64491092289
Earnings growth (YOY): 0.052503052503052504
Earnings growth average for industry: -0.092517543328727


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Forward Dividend & Yield: 0.50 (0.56%)
Altman Zscore: 1.8663013552911032
['COST', 'FTV', 'HAL', 'HFC', 'HUM', 'KSS', 'LLL', 'LYB', 'M', 'PM', 'TROW', 'WDC']


In [22]:
analyze("CTSH", industry_averages)
analyze("IPGP", industry_averages)
analyze("MNST", industry_averages)
analyze("PWR", industry_averages)
analyze("SWKS", industry_averages)
analyze("TIF", industry_averages)

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


open price
76.45


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


bvps
17.52865625


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ANALYSIS FOR TIF
Industry: Apparel, Accessories & Luxury Goods
Book Value: 31.6657811402
Price to book value ratio: 4.0608503997
Book value average for industry: 24.989656318172365
Current Ratio: 5.49572295806
Current ratio average for industry: 2.35684775922901
Debt Ratio: 0.405972824199
Debt ratio average for industry: 12.030265154602464
Working Capital Per Share (current assets - current liabilities) / num shares: 31.7661929208
Working Capital per share average for industry: 0.5493028866053118
Earnings growth (YOY): 0.8296346110737502
Earnings growth average for industry: 0.3561361430206693


C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Forward Dividend & Yield: 2.20 (1.70%)
Altman Zscore: 2.0561661461233847


['Apparel, Accessories & Luxury Goods',
 array([ 5.49572296,  5.64728192,  4.80668585,  5.3325228 ]),
 2.35684775922901,
 array([ 0.40597282,  0.40591651,  0.42801078,  0.44973555]),
 12.030265154602464,
 array([ 31.76619292,  23.78481596,  21.60989006,  21.24264023]),
 0.5493028866053118,
 array([ 31.66578114,  24.49331361,  22.78429834,  21.24189508]),
 4.0608503996960419,
 24.989656318172365,
 0.8296346110737502,
 0.3561361430206693,
 2.0561661461233847]

In [43]:
# add ROE, PE ratios, and free cash flow


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import tensorflow as  tf

In [49]:
financial_data = np.loadtxt(open("company_analysis.csv", "rb"), delimiter=",", skiprows=1, usecols=(1,2,3,4,5,6,7,8,9))
print(financial_data)

[[ 1.85729153  1.72004252  0.13724902 ...,  0.72676995  1.86057222  0.0283    ]
 [ 2.26065978  2.25188691  0.00877288 ..., -0.88033336  0.67138317  0.016     ]
 [ 1.27534403  2.40015127 -1.12480724 ..., -0.01566321  1.07855782  0.0479    ]
 ..., 
 [ 1.49503344  2.25188691 -0.75685347 ...,  4.7409691   0.69528284  0.0082    ]
 [ 0.03605323  0.06464109 -0.02858786 ..., -0.06396932 -0.95418052  0.0252    ]
 [ 3.85466179  2.40015127  1.45451052 ...,  0.1450206   1.86630136  0.0056    ]]


In [54]:
csv = open("company_analysis.csv", "r")
working_tickers = np.loadtxt(open("company_analysis.csv", "rb"), delimiter=",", skiprows=1, usecols=0, dtype=str)
prices = []
print(working_tickers)
for i in range(len(working_tickers)):
    prices.append(float(parse(working_tickers[i])['Open']))
        

['MMM' 'ABT' 'ABBV' 'ACN' 'ATVI' 'AAP' 'AES' 'AET' 'AMG' 'AFL' 'A' 'APD'
 'ALK' 'ALB' 'ARE' 'ALLE' 'AGN' 'ADS' 'LNT' 'ALL' 'MO' 'AEE' 'AAL' 'AEP'
 'AXP' 'AIG' 'AMT' 'AWK' 'AMP' 'ABC' 'AME' 'AMGN' 'APH' 'APC' 'ADI' 'ANTM'
 'AON' 'AOS' 'APA' 'AIV' 'AAPL' 'AMAT' 'APTV' 'ADM' 'ARNC' 'AJG' 'AIZ' 'T'
 'ADP' 'AVB' 'AVY' 'BHGE' 'BLL' 'BAC' 'BK' 'BAX' 'BBT' 'BDX' 'BBY' 'BLK'
 'HRB' 'BA' 'BWA' 'BXP' 'BMY' 'AVGO' 'BR' 'CHRW' 'CA' 'COG' 'CPB' 'COF'
 'CAH' 'CCL' 'CAT' 'CBOE' 'CBS' 'CNP' 'CTL' 'CF' 'SCHW' 'CVX' 'CB' 'CHD'
 'CI' 'XEC' 'CINF' 'CTAS' 'CSCO' 'C' 'CFG' 'CTXS' 'CLX' 'CME' 'CMS' 'KO'
 'CTSH' 'CL' 'CMCSA' 'CMA' 'CAG' 'COP' 'ED' 'STZ' 'COO' 'GLW' 'COST' 'COTY'
 'CCI' 'CSX' 'CMI' 'CVS' 'DHI' 'DHR' 'DRI' 'DE' 'DAL' 'XRAY' 'DVN' 'DLR'
 'DFS' 'DG' 'D' 'DOV' 'DWDP' 'DTE' 'DUK' 'DXC' 'EMN' 'ETN' 'ECL' 'EIX'
 'EMR' 'ETR' 'EOG' 'EQT' 'EFX' 'EQIX' 'EQR' 'ESS' 'EL' 'EVRG' 'ES' 'RE'
 'EXC' 'EXPE' 'EXPD' 'EXR' 'XOM' 'FAST' 'FRT' 'FDX' 'FIS' 'FITB' 'FE'
 'FLIR' 'FLS' 'FLR' 'FMC' 'FL' 'F' 'FTV' 'FBHS' 'BE

C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:843: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\kevin\Anaconda3\lib\site-packages\requests\packag

In [86]:
print(prices)
X, y = make_classification(n_samples=1000, n_features=9, n_informative=5, random_state=0, shuffle=False)
regr = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=0)
regr.fit(financial_data, prices)
print(regr.feature_importances_)
# cur_ratio, cur_ratio_av, cur_ratio_cmp_to_av, price_to_book, earning_growth, earning_growth_av, earning_growth_cmp_to_av, altman-zscore, div 
print(regr.predict([[1.85, 1.72, .13, 11.22, -0.04,-0.76, 0.726, 1.86, 0.0283 ]]))
print(regr.predict([[1.85, 1.72, .13, 11.22, -0.04, 1.6, 3.26, 2.86, 0.0283 ]]))
print(regr.predict([[2.0, 1, 1, 14, 2, 1, 1, 3, .03 ]]))

[193.66, 70.75, 84.18, 158.94, 69.95, 165.02, 14.74, 200.97, 119.37, 43.32, 65.8, 157.2, 64.12, 105.81, 122.29, 88.18, 165.17, 208.53, 43.03, 90.37, 63.6, 64.63, 36.59, 73.26, 104.93, 43.42, 152.82, 88.23, 130.48, 91.71, 72.46, 194.29, 90.33, 54.85, 88.32, 267.48, 156.95, 48.5, 37.01, 42.67, 209.55, 35.81, 78.13, 48.73, 21.44, 75.32, 97.75, 30.63, 141.12, 176.32, 92.99, 26.12, 47.62, 28.01, 48.0, 63.71, 49.59, 235.59, 73.89, 416.97, 26.54, 366.97, 41.31, 120.08, 52.46, 225.33, 117.07, 91.46, 44.44, 24.35, 37.85, 89.3, 52.91, 58.15, 127.05, 114.5, 57.36, 27.25, 20.7, 51.8, 46.86, 114.85, 126.62, 65.14, 216.76, 81.42, 78.94, 181.99, 45.72, 66.07, 37.79, 103.16, 153.69, 184.61, 48.59, 47.82, 69.12, 60.91, 38.09, 83.81, 35.48, 69.69, 76.1, 201.14, 264.46, 33.26, 229.81, 10.95, 109.25, 70.05, 143.16, 74.84, 37.3, 101.29, 105.96, 142.53, 56.5, 35.49, 32.99, 106.0, 70.48, 113.79, 70.4, 84.11, 58.68, 112.42, 83.41, 71.6, 82.21, 74.42, 154.05, 68.49, 70.67, 83.23, 106.15, 34.58, 105.04, 392.23,

In [76]:
(x_train, y_train, x_test, y_test) = train_test_split(financial_data, prices)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(20, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

AttributeError: module 'tensorflow' has no attribute 'keras'